In [1]:
from typing import Tuple

import flax.linen as nn
import jax
import jax.numpy as jnp
import jax.random
import jax.scipy.stats as jst
import tensorflow as tf
import tqdm
from sklearn import metrics

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
X_train = jnp.array(X_train.reshape(len(X_train), -1) / 255)
X_test = jnp.array(X_test.reshape(len(X_test), -1) / 255)

y_train = jax.nn.one_hot(y_train, 10)
y_test = jax.nn.one_hot(y_test, 10)

In [4]:
def init_mu(shape):
    return jnp.zeros(shape)


def init_rho(shape):
    sigma2 = jnp.ones(shape) * jnp.exp(-2)
    return jnp.log(jnp.exp(sigma2) - 1)

$$
f(w, \theta) = \log q(w | \theta) - \log p(w) - \log p(D | w)
$$



In [25]:

Theta = Tuple[jnp.ndarray, jnp.ndarray]
Params = Tuple[Theta, Theta, Theta]


def sample_w(mu: jnp.ndarray, rho: jnp.ndarray, rng_key: jnp.ndarray) -> jnp.ndarray:
    eps: jnp.ndarray = jax.random.normal(rng_key, mu.shape)
    w = mu + jnp.log(1 + jnp.exp(rho)) * eps
    return w


def bbb_mlp(params: Params, X: jnp.ndarray, rng_key: jnp.ndarray) -> Tuple[jnp.ndarray, Tuple[jnp.ndarray, ...]]:
    theta_W0, theta_b0, theta_W1, theta_b1, theta_W2, theta_b2 = params
    k0, k1, k2, k3, k4, k5 = jax.random.split(rng_key, 6)

    W0 = sample_w(*theta_W0, k0)
    b0 = sample_w(*theta_b0, k1)
    
    W1 = sample_w(*theta_W1, k2)
    b1 = sample_w(*theta_b1, k3)
    
    W2 = sample_w(*theta_W2, k4)
    b2 = sample_w(*theta_b2, k5)
    return nn.relu(nn.relu(X @ W0 + b0) @ W1 + b1) @ W2 + b2, (W0, W1, W2)


def z_score(w, mu, sigma):
    return (w - mu) / sigma


def kl_div(p: Params) -> jnp.ndarray:
    kl = jnp.array(0)
    
    mu_p, sigma_p = jnp.array(0), jnp.exp(-2)
    for (mu_q, rho_q) in p:
        sigma_q = jnp.log(1 + jnp.exp(rho_q))
        kl += jnp.sum(2 * jnp.log(sigma_p / sigma_q) - 1 + (sigma_q / sigma_p) ** 2 + ((mu_p - mu_q) / sigma_p) ** 2)
    return 0.5 * kl


@jax.jit
def train_step(
    params: Params,
    X: jnp.ndarray,
    y: jnp.ndarray,
    rng_key: jnp.ndarray,
    n_posterior_samples: int = 10,
    eta: float = 1e-3,
    beta: float = 0.05,
) -> Tuple[Params, jnp.ndarray, jnp.ndarray]:
    def loss_fn(p: Params, k: jnp.ndarray) -> jnp.ndarray:
        y_hat, _ = bbb_mlp(p, X, k)
        loss = (
            # log q(w | theta) / p(w)
            beta * kl_div(p)
            # log p(D | theta)
            - jnp.mean(
                jnp.sum(y * nn.log_softmax(y_hat, axis=-1), axis=-1)
            )
        )
        return loss

    G = jax.tree_map(lambda _: jnp.zeros_like(_), params)
    f = jax.value_and_grad(loss_fn)
    L = jnp.array(0)
    for i in range(n_posterior_samples):
        rng_key, key = jax.random.split(rng_key)
        l, g = f(params, key)
        L += l
        G = jax.tree_map(lambda c, k: c + k, g, G)

    # G = jax.tree_map(lambda g: g / n_posterior_samples, G)
    return update_params(params, G, eta=eta), L / n_posterior_samples, rng_key


def update_params(params: Params, gradients: Params, eta: float) -> Params:
    return jax.tree_map(lambda w, g: w - eta * g, params, gradients)


In [6]:
def get_batch_indices(rng: jnp.ndarray, dataset_size: int, batch_size: int) -> jnp.ndarray:
    steps_per_epoch = dataset_size // batch_size

    perms = jax.random.permutation(rng, dataset_size)
    perms = perms[:steps_per_epoch * batch_size]  # skip incomplete batch
    perms = perms.reshape((steps_per_epoch, batch_size))
    return perms

In [26]:
l0 = (28 ** 2, 512)
l1 = (512, 256)
l2 = (256, 10)

t0 = (init_mu(l0), init_rho(l0))
b0 = (init_mu(l0[-1]), init_rho(l0[-1]))

t1 = (init_mu(l1), init_rho(l1))
b1 = (init_mu(l1[-1]), init_rho(l1[-1]))

t2 = (init_mu(l2), init_rho(l2))
b2 = (init_mu(l2[-1]), init_rho(l2[-1]))

p0 = (t0, b0, t1, b1, t2, b2)

In [8]:
rng = jax.random.PRNGKey(0)

In [9]:
def with_beta(I):
    M = len(I)
    for ix, i in enumerate(I, 1):
        yield (2 ** (M - ix)) / (2 ** M - 1), i


In [27]:
r = rng
p = p0
# p = (W0, W1)
for e in range(50):
    ix = get_batch_indices(r, len(X_train), 128)
    I = tqdm.tqdm(ix, desc=f"Epoch {e}")
    for beta, i in with_beta(I):
        p, l, r = train_step(p, X_train[i], y_train[i], r, eta=1e-3, beta=beta)
        I.set_description(f"Epoch {e} (loss={l.item():.3f})")
        I.refresh()

Epoch 7 (loss=3.798):  17%|██████████████▎                                                                       | 78/468 [00:05<00:26, 14.72it/s]


KeyboardInterrupt: 

In [22]:
y_hat, _ = bbb_mlp(p, X_test, r)

In [23]:
print(
    metrics.classification_report(
        y_test.argmax(axis=1),
        y_hat.argmax(axis=1)
    )
)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.01      0.00      0.00      1135
           2       0.06      0.06      0.06      1032
           3       0.24      0.05      0.08      1010
           4       0.10      0.12      0.11       982
           5       0.10      0.42      0.16       892
           6       0.15      0.04      0.07       958
           7       0.09      0.08      0.09      1028
           8       0.03      0.02      0.02       974
           9       0.06      0.10      0.08      1009

    accuracy                           0.09     10000
   macro avg       0.08      0.09      0.07     10000
weighted avg       0.08      0.09      0.06     10000



In [24]:
metrics.roc_auc_score(y_test, nn.softmax(y_hat, axis=-1))

0.4743718184856297

In [14]:
def calculate_kl(p, w):
    kl = jnp.array(0)
    mu_p, sigma_p = jnp.array(0), jnp.exp(-2)
    for W, (mu_q, rho_q) in zip(w, p):
        sigma_q = jnp.log(1 + jnp.exp(rho_q))
        kl += jnp.sum(2 * jnp.log(sigma_p / sigma_q) - 1 + (sigma_q / sigma_p) ** 2 + ((mu_p - mu_q) / sigma_p) ** 2)
    return 0.5 * kl

In [16]:
_, w = bbb_mlp(p, X_test, rng)

In [18]:
kl_div(p, w)

Array(3495.823, dtype=float32)

In [19]:
calculate_kl(p, w)

Array(3549.7795, dtype=float32)